## I. Importing data

In [20]:
import pandas as pd
import numpy as np
data_x = pd.read_csv('traininginputs.csv')
data_y = pd.read_csv('trainingoutput.csv')

## II. Cleaning data 

In [21]:
data = pd.merge(data_x, data_y, on='PROC_TRACEINFO', how ='inner')

Nous explorant notre base de donnée afin de se faire une idée sur son contenu. Pour cela commencons par voir les dimensions de notre dataset (nombre d'observations et de variables):

In [22]:
nbr_lignes, nbr_colonnes = data.shape
print("Nbr de lignes :", nbr_lignes )
print("Nbr de colonnes :",nbr_colonnes )

Nbr de lignes : 34515
Nbr de colonnes : 15


Nous avons donc 14 variables pour 34.515 individus. 

Vérifions s'il y a des valeurs manquantes.

In [23]:
data.isna().sum()

PROC_TRACEINFO                         0
OP070_V_1_angle_value                  0
OP090_SnapRingPeakForce_value          0
OP070_V_2_angle_value                  0
OP120_Rodage_I_mesure_value            0
OP090_SnapRingFinalStroke_value        0
OP110_Vissage_M8_torque_value          0
OP100_Capuchon_insertion_mesure    18627
OP120_Rodage_U_mesure_value            0
OP070_V_1_torque_value                 0
OP090_StartLinePeakForce_value         0
OP110_Vissage_M8_angle_value           0
OP090_SnapRingMidPointForce_val        0
OP070_V_2_torque_value                 0
Binar OP130_Resultat_Global_v          0
dtype: int64

Il semblerait que la variable 'OP100_Capuchon_insertion_mesure' compte 18.627 valeurs manquantes soit pour 53% des observations. Dans ce cas il serait difficile et pas pertinant de remplacer ces valeurs manquantes. Deux options s'offrirons a nous : utiliser des modèles qui gèrent les valeurs manquantes ou tout simplement supprimer cette variable. Les deux options seront explorées.

In [24]:
type_col = data.dtypes
print(type_col)

PROC_TRACEINFO                      object
OP070_V_1_angle_value              float64
OP090_SnapRingPeakForce_value      float64
OP070_V_2_angle_value              float64
OP120_Rodage_I_mesure_value        float64
OP090_SnapRingFinalStroke_value    float64
OP110_Vissage_M8_torque_value      float64
OP100_Capuchon_insertion_mesure    float64
OP120_Rodage_U_mesure_value        float64
OP070_V_1_torque_value             float64
OP090_StartLinePeakForce_value     float64
OP110_Vissage_M8_angle_value       float64
OP090_SnapRingMidPointForce_val    float64
OP070_V_2_torque_value             float64
Binar OP130_Resultat_Global_v        int64
dtype: object


La variable PROC_TRACEINFO est sous format 'object' et n a aucun interet. L énnoncé confirme qu il s agit Id de pieces. Nous décidons donc de la supprimer:

In [25]:
data = data.drop('PROC_TRACEINFO', axis =1 )

OK toutes les variables sont quantitatives. 
Par contre notre prédicteur (Binar OP130_Resultat_Global_v) devrait etre une variable catégorielle a deux modalités et non une variable continue. Nous procédons donc au changement de son type.

In [26]:
data['Binar OP130_Resultat_Global_v'] = data['Binar OP130_Resultat_Global_v'].astype('category')
print(data['Binar OP130_Resultat_Global_v'].dtype)

category


## III. Data analysis 

L'analyse de donnée est un travail déterminant pour la bonne compréhension de nos données et permet parfois de mieux aiguiller le travail de modélisation. L'analyse de données permet d'étudier aussi bien la relation entre les différentes variables mais aussi de se faire une idée sur la population (exemple son homogénéité). Dans le cas ou l'on a une population hétérogène, il serait pertinant de créer plusieurs groupes d'observations, dans ce cas deux solutions sont envisageables : développer un modèle par groupe homoène d'invidus , ou encore rajouter une variable dans notre base de donnée qui porterait le groupe auquel appartiendrait chaque observation. L'inconvénient de cette stratégie est que pour chaque nouvelle observation , il faudra tout d'abord déterminer le groupe auquel elle appartient avant de l'intégrer dans le modèle.

In [27]:
print(data.describe())

       OP070_V_1_angle_value  OP090_SnapRingPeakForce_value  \
count           34515.000000                   34515.000000   
mean              159.906922                     156.915055   
std                15.662650                      11.271492   
min               101.800000                       0.000000   
25%               148.700000                     149.210000   
50%               158.000000                     156.180000   
75%               169.300000                     164.380000   
max               198.300000                     196.920000   

       OP070_V_2_angle_value  OP120_Rodage_I_mesure_value  \
count           34515.000000                 34515.000000   
mean              159.618236                   113.350222   
std                15.091490                     3.528522   
min                82.000000                    99.990000   
25%               149.400000                   111.040000   
50%               158.700000                   113.160000   
75%  

Contenu de la présence de valeurs manquantes dans l une de nos varibles nous décidons de partir un XGboost directement en premier essai car il intégre la possibilité de gestion de valeurs manquantes.

# IV. Machine learning modeling

#### 1. Diviser les données en train/test et définition des paramètres de la Validation croisée

In [28]:
import xgboost as xgb
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.metrics import make_scorer, roc_auc_score

Séparer les données en prédicteur (y) / variables prédictives (X) :

In [29]:
y = data['Binar OP130_Resultat_Global_v']
X = data.drop('Binar OP130_Resultat_Global_v', axis =1)

Nous séparons notre dataset en train/test en choisissant d'allouer 20% aux données test.  

In [30]:
X_train, X_test, y_train, y_test= train_test_split(X, y,test_size=0.2, random_state=123)

#### 2.1 modélisation avec XG Boost

In [31]:
# définition du modèle xgboost
model = xgb.XGBClassifier()

XGBoost, en tant qu'algorithme de boosting basé sur des arbres de décision, n'a  pas besoin de centrer ou de réduire les données en amont. Nous procédons donc au paramètrage de notre modèle. 

Comme évoqué précédemment, XGBoost est capable de gérer les valeurs manquantes automatiquement. Aucune configuration ne sera donc necessaire pour la gestion des valeurs manquantes.

#### 2.2 Recherche de paramètres
##### 2.2.1 Option 1: Selection manuelle (grille) des hyperparamètres

Deux types de paramètres: 
paramètres intervenant lors du calcul du gain /interet d ajouter un nouvel étage et donc structure de l arbre  & 
paramètres lors du calcul du poids optimal a chaque feuille d abre, impact au niveau l impact donc prédiction faite:

In [32]:
# Spécifiez la grille des hyperparamètres à explorer
param_grid = {
    'learning_rate': [ 0.05 ,0.06], # fraction de la correction appliquée
    'n_estimators': [50, 100, 120], # nombre d arbres a entrainer séquentiellement pour faire prédicteur final (biais /variance)
    'max_depth': [3, 4, 5], #profondeur maximale de ce que peut atteindre un arbre et donc des valeurs qu'il peut prendre soit 2^max_depth
    'min_child_weight': [1, 2, 3],
    'gamma': [ 0.1, 0.2], #définit le seuil du gain qu'apporte l'ajou de chaque nouveau noeud (0 -> pas de seuil)
    'subsample': [0.8, 0.9, 1.0],
    'colsample_bytree': [0.8, 0.9, 0.7],
}

# Créez un objet GridSearchCV pour la recherche par grille
grid_search = GridSearchCV(model, param_grid, scoring='roc_auc', cv=5, verbose=10, n_jobs=-1)

# Entraînez le modèle en utilisant la recherche par grille
grid_search.fit(X_train, y_train)

# Affichez les meilleurs hyperparamètres trouvés
print("Meilleurs hyperparamètres:", grid_search.best_params_)


Fitting 5 folds for each of 972 candidates, totalling 4860 fits


[CV 3/5; 1/972] START colsample_bytree=0.8, gamma=0.1, learning_rate=0.05, max_depth=3, min_child_weight=1, n_estimators=50, subsample=0.8
[CV 1/5; 1/972] START colsample_bytree=0.8, gamma=0.1, learning_rate=0.05, max_depth=3, min_child_weight=1, n_estimators=50, subsample=0.8
[CV 4/5; 1/972] START colsample_bytree=0.8, gamma=0.1, learning_rate=0.05, max_depth=3, min_child_weight=1, n_estimators=50, subsample=0.8
[CV 1/5; 2/972] START colsample_bytree=0.8, gamma=0.1, learning_rate=0.05, max_depth=3, min_child_weight=1, n_estimators=50, subsample=0.9
[CV 2/5; 1/972] START colsample_bytree=0.8, gamma=0.1, learning_rate=0.05, max_depth=3, min_child_weight=1, n_estimators=50, subsample=0.8
[CV 2/5; 2/972] START colsample_bytree=0.8, gamma=0.1, learning_rate=0.05, max_depth=3, min_child_weight=1, n_estimators=50, subsample=0.9
[CV 5/5; 1/972] START colsample_bytree=0.8, gamma=0.1, learning_rate=0.05, max_depth=3, min_child_weight=1, n_estimators=50, subsample=0.8
[CV 3/5; 2/972] START colsa

In [33]:
best_params = grid_search.best_params_ #enregistre les meilleurs paramètres trouvés.

best_model = xgb.XGBClassifier(**best_params)  # Remplacez XGBClassifier par le modèle que vous utilisez

best_model.fit(X_train, y_train) #entrainer le modèle avec les meilleurs paramètres trouvés.

y_pred_proba = best_model.predict_proba(X_test)[:, 1]  # prédire sur la partie X_test par vos données de test

roc_auc = roc_auc_score(y_test, y_pred_proba)  # Remplacez y_test par vos étiquettes de test

print("ROC AUC Score:", roc_auc)

ROC AUC Score: 0.7066849126907246


Avec cette approche nous arrivons au score 0.70 ROC_AUC ce qui est suppérieur au score de 0.675 obtenu par Valéo en utilisant une   classification naïve bayésienne. 

Pour l'obtention de ce score, les meilleurs hyperparamètres sont :  {'colsample_bytree': 0.8, 'gamma': 0.1, 'learning_rate': 0.06, 'max_depth': 4, 'min_child_weight': 1, 'n_estimators': 100, 'subsample': 1.0}

Néanmoins, l'inconvénient de l'approche 'brute force' est cette méthode soit biaisée par l'invention de l'humain. Le champs des paramètres possibles étant très large , le risque de tomber sur un 'minimal local' est aggravé par les limites de notre capacité calulatoire.

##### 2.2.2 Option 2: approche 'Halving Grid search'

Chercher des paramètres sur des échantillons de plus en plus important en écartant a chaque itération la moitiée des combinaisons les moins performantes.

##### 2.2.3 Option 3: approche 'Halving Grid search'

Permet d'explorer le champs des possibles sans forcément trouver la combinaison optimale. L'avantage que présente cette méthode est de pouvoir maitriser le temps de calcul.

##### 2.2.4 Option 4: approche par 'substitut'

##### 2.2.4.1 cas du processus gaussien (cherche bayesienne) avec Scikit optimize

L’idée  de l’optimisation bayésienne est de minimiser le nombre d’observations tout en convergeant rapidement vers la solution optimale. Concrétement l'algorithme déterminerait la prochaine configuration ayant le plus de potentiel à tester en fonction des résultats des itérations précédentes en fonction un processus gaussien.

Pour cela, il convient de connaître trois principes fondamentaux:
- Le processus Gaussien : d’exploiter les observations connues pour en déduire des probabilités d’événement qui n’ont pas encore été observées. Pour cela, il convient de déterminer pour chaque valeur X la distribution de probabilité. Le processus n'est pas applicable a toutes les observations a cause des limites de capacités calculatoires.

- Déterminer les points à plus fort potentiel avec la fonction d’acquisition : gagner en connaissance sur le comportement de la fonction et donc choisir une zone de l’espace de recherche où l’inconnu est grand : c’est l’exploration. D’autre part, nous souhaitons trouver le point qui minimise/maximise notre fonction : c’est l’exploitation. Ce compromis entre exploration et exploitation est exprimé par une fonction d’acquisition

- La fonction d’acquisition : Deux raisons permettent à une configuration donnée d’augmenter son potentiel, soit être dans une région loin de toutes configurations testées précédemment ou être dans une région près d’une configuration performante. En combinant ces deux critères, l’optimisation bayésienne cherche à réduire l’incertitude en explorant les régions peu explorées tout en exploitant les régions près d’une configuration performante. C’est ce qu’on appelle une fonction d’acquisition

In [34]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from skopt import BayesSearchCV
from skopt.space import Real, Integer
from pprint import pprint

In [35]:
model2 = xgb.XGBClassifier(
    n_jobs = -1,
    booster = 'gbtree', 
    objective = 'binary:logistic',
    eval_metric = 'auc', 
    enable_categorical = True, 
    verbosity = 1
)

In [36]:
# Define search space
search_spaces = {
     'learning_rate': np.arange(0.01, 1.0),
     'max_depth': np.arange(2, 20),
     'reg_lambda': np.arange(1e-9, 100),
     'reg_alpha': np.arange(1e-9, 100),
     'gamma': np.arange(1e-9, 0.5),  
     'n_estimators': np.arange(10, 5000)
}

In [42]:
# Create Bayesian CV for HP optimization
bayes_cv = BayesSearchCV(
                    estimator = model2,                                    
                    search_spaces = search_spaces,                      
                    scoring = 'roc_auc',                                  
                    cv = 5,                                   
                    n_iter = 300,                                      
                    n_points = 5,                                       
                    n_jobs = -1,                                        
                    refit=False,
                    verbose = 1,
                    random_state=42
)        

In [43]:
# Run bayesian CV
%time bayes_cv.fit(X_train, y_train)

Fitting 5 folds for each of 5 candidates, totalling 25 fits
Fitting 5 folds for each of 5 candidates, totalling 25 fits
Fitting 5 folds for each of 5 candidates, totalling 25 fits
Fitting 5 folds for each of 5 candidates, totalling 25 fits
Fitting 5 folds for each of 5 candidates, totalling 25 fits
Fitting 5 folds for each of 5 candidates, totalling 25 fits
Fitting 5 folds for each of 5 candidates, totalling 25 fits
Fitting 5 folds for each of 5 candidates, totalling 25 fits
Fitting 5 folds for each of 5 candidates, totalling 25 fits
Fitting 5 folds for each of 5 candidates, totalling 25 fits
Fitting 5 folds for each of 5 candidates, totalling 25 fits
Fitting 5 folds for each of 5 candidates, totalling 25 fits
Fitting 5 folds for each of 5 candidates, totalling 25 fits
Fitting 5 folds for each of 5 candidates, totalling 25 fits
Fitting 5 folds for each of 5 candidates, totalling 25 fits
Fitting 5 folds for each of 5 candidates, totalling 25 fits
Fitting 5 folds for each of 5 candidates

BayesSearchCV(cv=5,
              estimator=XGBClassifier(base_score=None, booster='gbtree',
                                      callbacks=None, colsample_bylevel=None,
                                      colsample_bynode=None,
                                      colsample_bytree=None, device=None,
                                      early_stopping_rounds=None,
                                      enable_categorical=True,
                                      eval_metric='auc', feature_types=None,
                                      gamma=None, grow_policy=None,
                                      importance_type=None,
                                      interaction_constraints=None,
                                      learning_rate=...
       5.6e+01, 5.7e+01, 5.8e+01, 5.9e+01, 6.0e+01, 6.1e+01, 6.2e+01,
       6.3e+01, 6.4e+01, 6.5e+01, 6.6e+01, 6.7e+01, 6.8e+01, 6.9e+01,
       7.0e+01, 7.1e+01, 7.2e+01, 7.3e+01, 7.4e+01, 7.5e+01, 7.6e+01,
       7.7e+01, 7.8e+01, 7.9e+01, 8.0e+01, 8.1e+01, 8.2e+01, 8.3e+01,
       8.4e+01, 8.5e+01, 8.6e+01, 8.7e+01, 8.8e+01, 8.9e+01, 9.0e+01,
       9.1e+01, 9.2e+01, 9.3e+01, 9.4e+01, 9.5e+01, 9.6e+01, 9.7e+01,
       9.8e+01, 9.9e+01])},
              verbose=1)

In [44]:
# Show best params
print('Best parameters:')
print("Best score = %.3f after %d runs" % (bayes_cv.best_score_, bayes_cv.n_iter))

Best parameters:
Best score = 0.638 after 300 runs


##### Best score = 0.630 after 20 runs

##### 2.2.4.2 SMAC - Forets aléatoires 


##### 2.2.4.3 Optimisation XG Boost - XG Boost

In [49]:
from ConfigSpace import ConfigurationSpace
from ConfigSpace.hyperparameters import CategoricalHyperparameter, UniformFloatHyperparameter, UniformIntegerHyperparameter

In [55]:
num_trees = UniformIntegerHyperparameter("num_trees", 10, 50, default_value =10)
max_features = UniformIntegerHyperparameter( "max_features", 1, 100, default_value =1)
min_weight_frac_leaf = UniformFloatHyperparameter("min_samples_to_split", 2,20, default_value =2)
min_samples_to_split = UniformIntegerHyperparameter("min_samples_to_split", 2,20, default_value=2)
min_samples_in_leaf = UniformIntegerHyperparameter("min_samples_in_leaf", 1,20, default_value =1)
max_leaf_nodes = UniformIntegerHyperparameter("max_leaf_nodes", 10,1000,default_value =100)

In [65]:
do_bootstrapping = CategoricalHyperparameter("do_bootstrapping", ["true", "false"], default_value="true")
criterion = CategoricalHyperparameter("Criterion", ["auc"], default_value="auc")

In [66]:
cs = ConfigurationSpace()
cs.add_hyperparameters([num_trees,min_weight_frac_leaf, min_samples_to_split, min_samples_in_leaf , max_leaf_nodes,  do_bootstrapping ])

HyperparameterAlreadyExistsError: Hyperparameter min_samples_to_split already exists in space.
Existing: min_samples_to_split, Type: UniformFloat, Range: [2.0, 20.0], Default: 2.0
New one: min_samples_to_split, Type: UniformInteger, Range: [2, 20], Default: 2Configuration space object:
  Hyperparameters:
    min_samples_to_split, Type: UniformFloat, Range: [2.0, 20.0], Default: 2.0
    num_trees, Type: UniformInteger, Range: [10, 50], Default: 10
